In [35]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time 
import tensorflow as tf
import numpy as np
from PIL import Image


In [36]:
def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass


def parse(serialized):
    '''
        Parse function required to read TFRecordDataset, "features" of the TFRecordDataset are passed through this function to read the Binary TFRecord file, that was
        created using the same "features". 
    '''
    features = {
                'label': tf.FixedLenFeature([],tf.int64),
                'image': tf.FixedLenFeature([],tf.string)
               }
    parsed_example = tf.parse_single_example(serialized, features)
    image_raw = parsed_example['image']

    # image data is in raw format obtained from TFRecordDataset, to use it as an input to the neural network it has to be decoded again in float32 format.

    image = tf.decode_raw (image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    label = parsed_example['label']
    return image, label

def load_tfrecord(train_file, validation_file, test_file):
    '''
    Reading TFRecords for Training, Validation and Test cases.
    '''
    print ('Reading TFRecord for training..')
    dataset = tf.data.TFRecordDataset(filenames = train_file)
    dataset = dataset.map(parse)
    total_training_samples = sum(1 for _ in tf.python_io.tf_record_iterator(train_file))
    print ('{} files has found in {}'.format(total_training_samples, train_file.split('/')[-1]))
    
    print ('Reading TFRecord for validation..')
    valdataset = tf.data.TFRecordDataset(filenames = validation_file)
    valdataset = valdataset.map(parse)
    total_validation_samples = sum(1 for _ in tf.python_io.tf_record_iterator(validation_file))
    print ('{} files has found in {}'.format(total_validation_samples, validation_file.split('/')[-1]))
    
    print ('Reading TFRecord for test..')
    testset = tf.data.TFRecordDataset(filenames = test_file)
    testset = testset.map(parse)
    total_test_samples = sum(1 for _ in tf.python_io.tf_record_iterator(test_file))
    print ('{} files has found in {}'.format(total_test_samples, test_file.split('/')[-1]))
    return (dataset, valdataset, testset, total_training_samples, total_validation_samples, total_test_samples)


In [25]:
def conv(inputs, filter, kernel, name, training, is_pool, pool_name = None):  
    '''
    Defining the Convolutional layer and pooling layer (if required) passing through the activation function ('ReLU' in this model).
    '''  
    conv = tf.layers.conv2d(inputs = inputs,
                            filters = filter,
                            kernel_size = [kernel, kernel],
                            padding='SAME',
                            trainable = training, 
                            kernel_initializer = tf.contrib.layers.variance_scaling_initializer(factor=1.0,mode='FAN_IN',uniform=True),
                            bias_initializer = tf.constant_initializer(0.0),
                            name = name, 
                            reuse = tf.AUTO_REUSE)       
        
    if is_pool:        
        conv = tf.layers.max_pooling2d(inputs = conv, 
                                        pool_size = [2, 2], 
                                        strides = 2,
                                        padding = 'VALID',
                                        name = pool_name)
    conv = tf.nn.leaky_relu(conv)    
    return conv


def fc(inputs, fc_size, name, training, is_dropout, keep_prob = 1.0, activation = None, name_dropout = None):
    '''
    Defining the Fully Connected Layer and Dropout Layer(if required). Activation function can be passed if required.
    '''
    
    fc = tf.layers.dense (  inputs, 
                            fc_size, 
                            activation = activation, 
                            kernel_initializer = tf.contrib.layers.variance_scaling_initializer(factor=1.0,mode='FAN_IN',uniform=True),
                            bias_initializer = tf.constant_initializer(0.0),
                            name = name, 
                            trainable = training, 
                            reuse = tf.AUTO_REUSE)
#     if is_dropout:
        
#         fc = tf.layers.dropout( tf.nn.relu(fc), 
#                                 keep_prob,
#                                 training = training, 
#                                 name = name)
        
    return fc


In [1]:
class ConvNet(object):

    '''
    ConvNet here is the Alexnet model (introduced in 2011), Having 5 Convolutional Layers and 3 Fully Connected Layers. This model uses Training set, Validation set 
    and Test set for Training purpose. Initial variables are defined in the __init__ function. 

    Note: Don't try to change these variables when using already trained model. In case of training purpose only try to change it.

    Note: self.image_size = 224 (used for training), if the input size of the image(for prediction) is not as same as (224,224,3) an error will occur.

    Predict function has been definied inside this class, thus, this model is "Freezed" (meaning self.training = False, weights and all other parameters/variables 
    will not change.) 

    Note: As the model is freezed, it can not be used to optimize model further. (that is why 'self.optimize' has been commented in the 'build' function)
    '''

    def __init__(self):
        self.lr = 0.003
        self.batch_size = 4096
        self.keep_prob1 = 0.50
        self.keep_prob2 = 0.50
        self.fc_size1 = 128
        self.fc_size2 = 128
        self.filter1 = 16
        self.filter2 = 16
        self.filter3 = 16
        self.filter4 = 64
        self.filter5 = 64
        self.kernel1 = 3
        self.kernel2 = 3
        self.kernel3 = 3
        self.kernel4 = 3
        self.kernel5 = 3
        self.training = True #If you want to train the model, change it as "True" and uncomment "self.optimize" inside 'build' function
        self.image_size = 53 #All the inputs should be of size (224,224,3). To use different image size, train the model again with different image size to use that particular image size for prediction
        self.num_channels = 3
        self.n_classes = 2 # [0: Food, 1: Drink, 2: Other]
        self.buffer_size = 2048

        ''' Location of the Training, Validation and Test TFRecord files'''
        
        self.train_filename = 'tfrecord_dataset/tfrecord_53_2_79_A_O_train'
        self.val_filename = 'tfrecord_dataset/tfrecord_53_2_79_A_O_val'
        self.test_filename = 'tfrecord_dataset/tfrecord_53_2_79_A_O_test'

        ''' Location of the TFRecord file that will be created for prediction, if 'folder_location' is passed when calling 'predict' function.'''        
        
        self.tfrecord_pred = 'tfrecords/tfrecord_224_pred'
        
    
    def get_data(self):

        with tf.name_scope('data'):

            ''' Preprocessing the dataset before passing it as an input to the model.'''
            
            train_data, val_data, test_data, self.n_train, self.n_val, self.n_test = load_tfrecord(self.train_filename, self.val_filename, self.test_filename)
            
            train_data = train_data.shuffle(buffer_size = self.buffer_size)
            train_data = train_data.batch(self.batch_size)
    
            val_data = val_data.shuffle(buffer_size = self.buffer_size)
            val_data = val_data.batch(self.batch_size)

            test_data = test_data.batch(self.batch_size)

            #Creating iterator to iterate the batches of images along with its labels, from structure of this dataset.
            
            iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                                   train_data.output_shapes)
            img, self.label = iterator.get_next()

            # Model takes shape [?, image size, image size, number of channels] as an input, i.e Reshaping is required. 
            self.img = tf.reshape(img, shape=[-1, self.image_size, self.image_size, self.num_channels])

            # Initializer for Train, Validate and Test datasets.
            self.train_init = iterator.make_initializer(train_data)  
            self.val_init = iterator.make_initializer(val_data)  
            self.test_init = iterator.make_initializer(test_data)

    def inference(self):

        #standardizing all (whole batch of images) the input images before passing it inside the model.
        self.image = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), self.img)

        #Five Convolution Layers
        conv1 = conv(self.image, self.filter1, self.kernel1, 'conv1', self.training, True, 'pool1')
        conv2 = conv(conv1, self.filter2, self.kernel2, 'conv2', self.training, True, 'pool2')
        conv3 = conv(conv2, self.filter3, self.kernel3, 'conv3', self.training, False)
        conv4 = conv(conv3, self.filter4, self.kernel4, 'conv4', self.training, False)
        conv5 = conv(conv4, self.filter5, self.kernel5, 'conv5', self.training, True, 'pool3')
        
        #Flattening the outcome.
        layer_shape = conv5.get_shape()
        num_features = layer_shape[1:4].num_elements()
        conv5 = tf.reshape(conv5, [-1, num_features])

        #Three Fully Connected Layers
        fc1 = fc(conv5, self.fc_size1, 'fc1', self.training, True, self.keep_prob1, 
                 tf.nn.leaky_relu, 'dropout1')
        fc2 = fc(fc1, self.fc_size2, 'fc2', self.training, True, self.keep_prob2,
                tf.nn.leaky_relu, 'dropout2')
        
        #Third FC layer's outcomes must be equal to the number of classes for classification.
        self.logits = fc(fc2, self.n_classes, 'logits', self.training, False)

        self.y_pred = tf.nn.softmax(self.logits)
        self.y_pred_cls = tf.argmax(self.y_pred, axis=1)

    def loss(self):  
        '''
        Loss Function that need to be optimized.
        '''      
        with tf.name_scope('loss'):            
            entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels= tf.one_hot(self.label,self.n_classes), logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')
    
    def optimize(self):  
        '''
        Optimizing the Loss Function using AdamOptimizer. 
        '''      
        opt = tf.train.AdamOptimizer(self.lr)
        self.opt = opt.minimize(self.loss, global_step = self.gstep)

    def summary(self):
        ''' This function is a summary writer, use TensorBoard to have a look on the variations of loss, accuracy and histogram loss. '''
        with tf.name_scope('summaries'):            
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('histogram loss', self.loss)
            self.summary_op = tf.summary.merge_all()
    
    def eval(self):
        ''' To check the training accuracy this function is used. When correct labels are given, after comparing it with predicted labels, 
        this function gives number of correct predictions for that passed batch and it alse calculates the confusion matrix for that batch.
        '''
        with tf.name_scope('predict'):
            correct_preds = tf.equal(self.y_pred_cls, self.label)
            self.matrix = tf.confusion_matrix(self.label, self.y_pred_cls, num_classes=self.n_classes)
            self.accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

    def pred(self):
        self.training = False
        self.prediction = self.y_pred_cls

    def build(self):
        ''' 
        Build the model first before starting either training or prediction.
        '''
        tf.reset_default_graph()
        self.gstep = tf.Variable(0, dtype = tf.int32 , trainable=False, name='global_step')
        self.get_data()
        self.inference()
        self.loss()
        
        ''' Uncomment self.optimize in case of training'''

        self.optimize()
        self.eval()
        self.summary()

        
    def build1(self):
        ''' 
        Build the model first before starting either training or prediction.
        '''
        tf.reset_default_graph()
        self.gstep = tf.Variable(0, dtype = tf.int32 , trainable=False, name='global_step')
        self.get_data()
        self.inference()
        self.loss()
        self.eval()
        self.summary()


    def train_one_epoch(self, sess, saver, init, writer, epoch, step):
        ''' Returns Total Loss for that epoch after training.'''

        #initialize the initializer (for training or validation or testing) so when 'iterator.get_next()' calls, it gives batches of images from Training or Validation or Test set.
        sess.run(init) 
        
        self.training = True
        
        total_loss = 0
        try:
            while True:
                _, l, summaries = sess.run([self.opt, self.loss, self.summary_op])
                writer.add_summary(summaries, global_step=step)
                step += 1
                total_loss += l
        except tf.errors.OutOfRangeError:
            pass
        saver.save(sess, 'checkpoints/convnet_layers/alexnet-convnet', step)
        
        return step, total_loss

    def validation_loss(self, sess, init):
        ''' Returns Validation loss, As we don't want variables to be in Training state, 'self.training = False' while calculating Validation Loss.'''

        sess.run(init)
        validation_loss = 0
        self.training = False
        try:
            while True:
                l = sess.run([self.loss])
                validation_loss += l[0]
        except tf.errors.OutOfRangeError:
            pass
        return validation_loss

    def eval_once(self, sess, init, writer, epoch, step, training):
        ''' To get the Total Correct Prediction of the whole dataset not for just a batch, we call the 'self.accuracy' on the entire dataset.'''
        start_time = time.time()
        sess.run(init)
        self.training = training
        total_correct_preds = 0
        try:
            while True:                
                accuracy_batch, summaries = sess.run([self.accuracy, self.summary_op])            
                writer.add_summary(summaries, global_step=step)
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass
        return total_correct_preds

    def con_matrix(self, sess, init):
        ''' Takes all the confusion matrices for every batch passed and sum it up to get the confusion matrix of the whole dataset passed inside the model.'''
        sess.run(init)  
        self.training = False      
        matrix = np.zeros((self.n_classes,self.n_classes))
        try:
            while True:
                matrix = matrix + sess.run([self.matrix])[0] 
        except tf.errors.OutOfRangeError:
            pass                
        return matrix    

    def train(self, n_epochs):
        ''' Function to be called when we want to train the model. number of epochs needs to be entered when calling this function.'''

        #creating Checkpoints Directory if not already exists.
        safe_mkdir('checkpoints')
        safe_mkdir('checkpoints/convnet_layers')
        writer = tf.summary.FileWriter('./graphs/convnet_layers', tf.get_default_graph())
        gstep = tf.Variable(0.0 , trainable=False, name='global_step')

        config = tf.ConfigProto(allow_soft_placement = True)
        config.gpu_options.allow_growth = True
        config.gpu_options.allocator_type = 'BFC'
        with tf.Session(config = config).as_default() as sess: 
            
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()

            #Restoring from saved checkpoint if it exists. 
            '''
            Note: If want to train the model from scratch then delete all the files inside 'Checkpoint/connet_layers' so that model does not restore from the checkpoint.
            '''
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_layers/checkpoint'))
#             try:
#                 if ckpt and ckpt.model_checkpoint_path:
#                     saver.restore(sess, ckpt.model_checkpoint_path)
#             except:
#                 pass   
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)

            step = self.gstep.eval()

            for epoch in range(n_epochs):
                start_time = time.time()
                step, total_loss = self.train_one_epoch(sess, saver, self.train_init, writer, epoch, step)
                total_validation_loss = self.validation_loss(sess, self.val_init)
                total_corr_train = self.eval_once(sess, self.train_init, writer, epoch, step, training = True)
                total_corr_val = self.eval_once(sess, self.val_init, writer, epoch, step, training = False)
                end_time =time.time()
                print_statement = 'Epoch {0} --- Accuracy: {1:>6.1%}  V_Accuracy: {2:>6.1%}  Loss/epoch: {3}   V_Loss/epoch: {4}  Time: {5} sec'
                print(print_statement.format(epoch, 
                                            total_corr_train/self.n_train, 
                                            total_corr_val/self.n_val, 
                                            round(total_loss, 3), 
                                            round(total_validation_loss,3),
                                            round(end_time-start_time, 2)))
                
                #Printing Confustion matrix after the last epoch.
                if (epoch == n_epochs -1):
                    matrix = self.con_matrix(sess, self.test_init)
                    print ('Confusion Matrix: \n', matrix)

            #Saving the model to the given location.
            save_path = saver.save(sess, "Saver/model.ckpt")
                    
            #Running the model on the test set.
            test_accuracy = self.eval_once(sess, self.test_init, writer, epoch = 1, step = 1, training = False) 
            test_loss = self.validation_loss(sess, self.test_init)

            print ('Test Accuracy: {0:>6.1%}    Test Loss: {1}'. format(test_accuracy/self.n_test, round(test_loss, 3)))          
            
                
        writer.close()

    def predict(self, flag, image_location = None, folder_location = None, tfrecord_location = None):

        '''
        Once model is trained, we can use predict function to predict the outcome of the image. This function takes 
        1. Single Image
        2. Folder Location containing images
        3. Already created TFRecord File

        If input is a single image, model will not creating any TFRecord FIle and gives the output, giving image directly as an input. 
        (Shape of the input image must be the same as shape of image the model was originally trained on.)

        If folder location is given, model first creates a TFRecord File and then reads the created TFRecord File and gives the outcome.
        '''
        
        def _bytes_feature(value):
            return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
        
        def get_image_binary1(filename):
            try:
                image = Image.open(filename)
                image = image.resize((self.image_size, self.image_size), Image.ANTIALIAS)
                image = np.asarray(image, np.uint8)
                image = image.tobytes()

                if (len(image) == self.image_size*self.image_size*self.num_channels):
                    return image
            except exception as e:
                print ('Image Skipped!!!!!!!!!!!!! ...Error while reading "{}" '.format(filename))
                pass


        def parse1(serialized):
            features = {
                        'image' : tf.FixedLenFeature([], tf.string),
                        'image_name': tf.FixedLenFeature([],tf.string)
                       }
            parsed_example = tf.parse_single_example(serialized, features)
            image_raw = parsed_example['image']
            image_name = parsed_example['image_name']
            image = tf.decode_raw (image_raw, tf.uint8)
            image = tf.cast(image, tf.float32)
            image = tf.reshape(image, shape=[self.image_size,self.image_size,self.num_channels]) 
            return image, image_name

        def load_tfrecord1(predict_file):
            tfrecord_name = predict_file.split('/')[-1]
            print ('Reading {} for prediction'.format(tfrecord_name))
            dataset = tf.data.TFRecordDataset(filenames = predict_file)
            dataset = dataset.map(parse1)
            total_prediction_examples = sum(1 for _ in tf.python_io.tf_record_iterator(predict_file))
            print ('{} files has found in {}'.format(total_prediction_examples, tfrecord_name))
            return (dataset, total_prediction_examples, tfrecord_name)

        def write_tfrecord1(image, image_name):

            a = get_image_binary1(image)
            
            example = tf.train.Example(features=tf.train.Features(feature = 
                                                                      {                                                              
                                                                          'image': _bytes_feature(a),
                                                                          'image_name' : _bytes_feature(bytes(image_name))                                                            
                                                                      }))
            return example
        
        def score(array):
            score = np.zeros((array.shape[0]))
            for n, image in enumerate(array):
                if (image[2]>image[1] and image[2]>image[0]):
                    score[n]=2
                elif (image[1]>image[2] and image[1]>image[0]):
                    score[n]=1
                elif (image[0]>image[1] and image[0]>image[2]):
                    score[n]=0
                elif(image[0]>image[1] and image[0]==image[2]):
                    score[n]=2
                elif (image[0]>image[2] and image[0]==image[1]):
                    score[n]=1
                elif (image[1]>image[0] and image[1]==image[2]):
                    score[n]=2       
                else:
                    score[n]=2
            return score  
        
        def score1(array):
            f = 0
            d = 0
            o = 0
            for value in array:
                if value==0:
                    d+=1
                elif value==1:
                    f+=1
                else:
                    o+=1
            print (d,f,o)
        
        
        if folder_location != None:
            for docid in os.listdir(folder_location):            
                print ('creating TFRecord from {}'.format(docid))
                tfrecord_pred = 'tfrecord_test/'+docid
                print (tfrecord_pred)
                writer = tf.python_io.TFRecordWriter(tfrecord_pred)
                for images in os.listdir(os.path.join(folder_location, docid)):
                    try:
                        image_location = os.path.join(os.path.join(folder_location, docid), images)
                        #print (image_location, images)
                        example = write_tfrecord1(image_location, images) 
                        print (type(example))
                        writer.write(example.SerializeToString())
                    except Exception as e:
                        print ('Image Skipped! ...Error while reading "{}" '.format(images))
                        pass
                writer.close()                
   
        if tfrecord_location == None:
            tfrecord_location = 'tfrecord_test/'
        p = []
        pd = []
        for tfrecords in os.listdir(tfrecord_location):
            prediction_data, n_prediction, docID = load_tfrecord1(os.path.join(tfrecord_location, tfrecords))
            prediction_data = prediction_data.batch(self.batch_size)
            iterator1 = tf.data.Iterator.from_structure(prediction_data.output_types, 
                                                       prediction_data.output_shapes) 
#             config = tf.ConfigProto(allow_soft_placement = True)
#             config.gpu_options.allow_growth = True
#             config.gpu_options.allocator_type = 'BFC'
#             with tf.Session(config = config).as_default() as sess: 
            if n_prediction > 0 :
                with tf.Session().as_default() as sess: 
                    sess.run(tf.global_variables_initializer())
                    saver = tf.train.Saver()
                    #self.training = False                
                    sess.run(iterator1.make_initializer(prediction_data))
                    data = iterator1.get_next()
                    pred = np.zeros(n_prediction)
                    n = 0
                    details = []
                    try:
                        while True:
                            image = sess.run(data)                          
                            new_pred = np.zeros((n_prediction, 3))
                            #new_pred1 = np.zeros((n_prediction, 3))                
                            if (flag == 0):
                                for savers in os.listdir('RESULT/FD_Other'):
                                    saver_location = os.path.join('RESULT/FD_Other', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                            elif (flag==2):
                                for savers in os.listdir('RESULT/O_A'):
                                    saver_location = os.path.join('RESULT/O_A', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                            elif (flag==3):
                                for savers in os.listdir('RESULT/DO'):
                                    saver_location = os.path.join('RESULT/DO', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                            elif (flag==4):
                                for savers in os.listdir('RESULT/ADO'):
                                    saver_location = os.path.join('RESULT/ADO', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                            else:
                                for savers in os.listdir('RESULT/FDO'):
                                    saver_location = os.path.join('RESULT/FDO', savers)
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]}) 
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                            for i in range(len(image[1])):
                                result = []
                                result.append(docID)
                                result.append(image[1][i].decode('utf-8'))
                                pd.append(result)
                    except tf.errors.OutOfRangeError:
                        pass
                    #print (pred)
                    score1(pred)
                    for n in pred:
                        p.append(int(n))
        
        return (p,pd)
    
    
    def predict1(self, flag, folder_location = None, tfrecord_location = None, image_location = None):
        
        def _bytes_feature(value):
            return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
        
        def get_image_binary1(filename):
            try:
                image = Image.open(filename)
                image = image.resize((self.image_size, self.image_size), Image.ANTIALIAS)
                image = np.asarray(image, np.uint8)
                image = image.tobytes()

                if (len(image) == self.image_size*self.image_size*self.num_channels):
                    return image
            except exception as e:
                print ('Image Skipped! ...Error while reading "{}" '.format(filename))
                pass


        def parse1(serialized):
            features = {
                        'image' : tf.FixedLenFeature([], tf.string),
                        'image_name': tf.FixedLenFeature([],tf.string)
                       }
            parsed_example = tf.parse_single_example(serialized, features)
            image_raw = parsed_example['image']
            image_name = parsed_example['image_name']
            image = tf.decode_raw (image_raw, tf.uint8)
            image = tf.cast(image, tf.float32)
            image = tf.reshape(image, shape=[self.image_size,self.image_size,self.num_channels]) 
            return image, image_name

        def load_tfrecord1(predict_file):
            tfrecord_name = predict_file.split('/')[-1]
            print ('Reading {} for prediction'.format(tfrecord_name))
            dataset = tf.data.TFRecordDataset(filenames = predict_file)
            dataset = dataset.map(parse1)
            total_prediction_examples = sum(1 for _ in tf.python_io.tf_record_iterator(predict_file))
            print ('{} files has found in {}'.format(total_prediction_examples, tfrecord_name))
            return (dataset, total_prediction_examples, tfrecord_name)

        def write_tfrecord1(image, image_name):

            a = get_image_binary1(image)
            #print (len(a))
            #print (image_name)
            example = tf.train.Example(features=tf.train.Features(feature = 
                                                                      {                                                              
                                                                          'image': _bytes_feature(a),
                                                                          'image_name' : _bytes_feature(bytes(image_name))                                                            
                                                                      }))
            return example
        
        def score(array):
            score = np.zeros((array.shape[0]))
            for n, image in enumerate(array):
                if (image[2]>image[1] and image[2]>image[0]):
                    score[n]=2
                elif (image[1]>image[2] and image[1]>image[0]):
                    score[n]=1
                elif (image[0]>image[1] and image[0]>image[2]):
                    score[n]=0
                elif(image[0]>image[1] and image[0]==image[2]):
                    score[n]=2
                elif (image[0]>image[2] and image[0]==image[1]):
                    score[n]=1
                elif (image[1]>image[0] and image[1]==image[2]):
                    score[n]=2       
                else:
                    score[n]=2
            return score  
        
        def score1(array):
            f = 0
            d = 0
            o = 0
            for value in array:
                if value==0:
                    d+=1
                elif value==1:
                    f+=1
                else:
                    o+=1
            print (d,f,o)
            
        if folder_location != None:
            docid = folder_location.split('/')[-1]
            print ('creating TFRecord from {}'.format(docid))
            tfrecord_pred = 'tfrecord_test/'+docid
            print (tfrecord_pred)
            writer = tf.python_io.TFRecordWriter(tfrecord_pred)
            for images in os.listdir(folder_location):
                try:
                    image_location = os.path.join(folder_location, images)
                    example = write_tfrecord1(image_location, images) 
                    #print (type(example))
                    writer.write(example.SerializeToString())
                except Exception as e:
                    print ('Image Skipped! ...Error while reading "{}" '.format(images))
                    pass
            writer.close() 
            
            
        try:    
            prediction_data, n_prediction, docID = load_tfrecord1(tfrecord_location)
            if n_prediction == 0:
                return None
        except:
            return None
        prediction_data = prediction_data.batch(self.batch_size)
        iterator1 = tf.data.Iterator.from_structure(prediction_data.output_types, 
                                                   prediction_data.output_shapes) 

        if n_prediction > 0 :
            with tf.Session().as_default() as sess: 
                sess.run(tf.global_variables_initializer())
                saver = tf.train.Saver()              
                sess.run(iterator1.make_initializer(prediction_data))
                data = iterator1.get_next()
                pred = np.zeros(n_prediction)
                n = 0
                details = []
                p = []
                try:
                    while True:
                        image = sess.run(data)                          
                        new_pred = np.zeros((n_prediction, 3))               
                        if (flag == 0):
                            for savers in os.listdir('RESULT/FD_Other'):
                                saver_location = os.path.join('RESULT/FD_Other', savers)                                
                                saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                score1(topk)
                                for i,prediction in enumerate(topk):
                                    new_pred[i][prediction] += 1
                                #print (topk)
                            s = score(new_pred)
                            for i,value in enumerate(s):
                                i += n
                                pred[i]=value
                            n += len(topk)
                            print (pred)
                        elif (flag==2):
                            for savers in os.listdir('RESULT/O_A'):
                                saver_location = os.path.join('RESULT/O_A', savers)                                
                                saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                score1(topk)
                                for i,prediction in enumerate(topk):
                                    new_pred[i][prediction] += 1
                                #print (topk)
                            s = score(new_pred)
                            for i,value in enumerate(s):
                                i += n
                                pred[i]=value
                            n += len(topk)
                            print (pred)
                        elif (flag==3):
                                for savers in os.listdir('RESULT/DO'):
                                    saver_location = os.path.join('RESULT/DO', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                        elif (flag==4):
                                for savers in os.listdir('RESULT/ADO'):
                                    saver_location = os.path.join('/home/justdial/jd/RESULT/ADO', savers)                                
                                    saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                    topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]})
                                    score1(topk)
                                    for i,prediction in enumerate(topk):
                                        new_pred[i][prediction] += 1
                                    #print (topk)
                                s = score(new_pred)
                                for i,value in enumerate(s):
                                    i += n
                                    pred[i]=value
                                n += len(topk)
                                print (pred)
                        else:
                            for savers in os.listdir('RESULT/FDO'):
                                saver_location = os.path.join('RESULT/FDO', savers)
                                saver.restore(sess, os.path.join(saver_location,'model.ckpt'))                                     
                                topk = sess.run(self.y_pred_cls, feed_dict = {self.img: image[0]}) 
                                score1(topk)
                                for i,prediction in enumerate(topk):
                                    new_pred[i][prediction] += 1
                                #print (topk)
                            s = score(new_pred)
                            for i,value in enumerate(s):
                                i += n
                                pred[i]=value
                            n += len(topk)
                            print (pred)
                        for i in range(len(image[1])):
                            result = []
                            result.append(docID)
                            result.append(image[1][i].decode('utf-8'))
                            details.append(result)
                except tf.errors.OutOfRangeError:
                    pass
                #print (pred)
                score1(pred)
                for n in pred:
                    p.append(int(n))

                return (p,details)
        

            
            

In [43]:
if __name__ == '__main__':
    model = ConvNet()
    model.build()
    model.lr = 0.0001
    model.train(n_epochs = 1)
    #model.predict(tfrecord_location = 'tfrecord_test/ambience')

Reading TFRecord for training..
19632 files has found in tfrecord_53_2_79_A_O_train
Reading TFRecord for validation..
2725 files has found in tfrecord_53_2_79_A_O_val
Reading TFRecord for test..
2486 files has found in tfrecord_53_2_79_A_O_test
INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
INFO:tensorflow:Restoring parameters from checkpoints/convnet_layers/alexnet-convnet-220
Epoch 0 --- Accuracy:  86.1%  V_Accuracy:  82.7%  Loss/epoch: 1.505   V_Loss/epoch: 0.404  Time: 14.11 sec
Confusion Matrix: 
 [[1084.  166.]
 [ 280.  956.]]
Test Accuracy:  82.1%    Test Loss: 0.409


In [42]:
model = ConvNet()
model.training = False
model.build1()
for location in os.listdir('/home/justdial/jd/tfrecord_test'):
#     folder_location = os.path.join('/home/justdial/jd/Manyata/Dataset/try/', location)
    tfrecord_location = os.path.join('tfrecord_test', location)
    try:
        a,l1 = model.predict1(3, tfrecord_location= tfrecord_location)
    except:
        pass

Reading TFRecord for training..
20200 files has found in tfrecord_53_3_80_DAO_train
Reading TFRecord for validation..
2526 files has found in tfrecord_53_3_80_DAO_val
Reading TFRecord for test..
2526 files has found in tfrecord_53_3_80_DAO_test
INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
Reading other(no_ambience) for prediction
41 files has found in other(no_ambience)
INFO:tensorflow:Restoring parameters from /home/justdial/jd/RESULT/ADO/Saver1/model.ckpt
(6, 10, 25)
[1. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 2. 0. 2. 1. 0. 2. 0. 2. 2.
 2. 2. 1. 2. 2. 2. 1. 1. 1. 2. 2. 1. 1. 2. 0. 0. 2.]
(6, 10, 25)
Reading others for prediction
1585 files has found in others
INFO:tensorflow:Restoring parameters from /home/justdial/jd/RESULT/ADO/Saver1/model.ckpt
(531, 496, 558)
[2. 2. 1. ... 2. 2. 1.]
(531, 496, 558)
Reading drinks for prediction
2578 files has found in drinks
INFO:tensorflow:Restoring parameters from /home/justdial/jd/RESULT/ADO/Saver1/mo

In [34]:
model = ConvNet()
model.training = False
model.n_classes = 2
model.build1()
a,l1 = model.predict(0)
# a, l1 = model.predict(0, folder_location = '/home/justdial/jd/Manyata/Dataset/try/test_amb_dataset')

model1 = ConvNet()
model1.n_classes = 2
model1.training = False
model1.build1()
b,l2 = model1.predict(2)

model2 = ConvNet()
model2.n_classes = 3
model2.training = False
model2.build1()
c,l3 = model2.predict(1)

model3 = ConvNet()
model3.training = False
model3.n_classes = 2
model3.build1()
d,l4 = model3.predict(3)

model4 = ConvNet()
model4.n_classes = 3
model4.training = False
model4.build1()
e,l5 = model4.predict(4)

assert l1==l2
assert l2==l3
assert l3==l4
assert l3==l5
pred = np.zeros(len(a))
for i in range(len(a)):
    if b[i]==1 and e[i]==2 and a[i]==1  and c[i]==2:
        pred[i]=3
    elif a[i]==0 and c[i]==1:
        pred[i]=1
    elif d[i] == 0 and a[i]==0 and e[i]==1 :
        pred[i]=0
    else:
        pred[i]=2
        
result = []
for i in range(len(a)):
    p = []
    p.append(l1[i][0])
    p.append(l1[i][1])
    p.append(int(pred[i]))
    result.append(p)

Reading TFRecord for training..
20670 files has found in tfrecord_53_2_80_A_O_train
Reading TFRecord for validation..
2588 files has found in tfrecord_53_2_80_A_O_val
Reading TFRecord for test..
2588 files has found in tfrecord_53_2_80_A_O_test
INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
Reading others for prediction
1585 files has found in others
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_70_91T_91V/model.ckpt
793 792 0
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_70_88V_88T/model.ckpt
620 965 0
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_80_91T_90V/model.ckpt
649 936 0
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_80_92V_91T/model.ckpt
692 893 0
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_70_90V_89T/model.ckpt
602 983 0
INFO:tensorflow:Restoring parameters from RESULT/FD_Other/Saver_FD_Other_80_91V_91T

21 20 0
[0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
21 20 0
Reading other(nodrink) for prediction
83 files has found in other(nodrink)
INFO:tensorflow:Restoring parameters from RESULT/O_A/Saver_OA_80_81V_82T/model.ckpt
65 18 0
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
65 18 0
Reading test2 for prediction
100 files has found in test2
INFO:tensorflow:Restoring parameters from RESULT/O_A/Saver_OA_80_81V_82T/model.ckpt
56 44 0
[0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.

3 37 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_80_81V_83T/model.ckpt
12 28 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_80_83V_84T/model.ckpt
11 29 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_90_84V_83T/model.ckpt
7 33 0
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1.]
5 35 0
Reading other(no_ambience) for prediction
41 files has found in other(no_ambience)
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_70_81V_82T/model.ckpt
2 39 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_80_81V_83T/model.ckpt
5 36 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_80_83V_84T/model.ckpt
6 35 0
INFO:tensorflow:Restoring parameters from RESULT/DO/Saver_DO_90_84V_83T/model.ckpt
7 34 0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2 39 0
Reading other(n

In [50]:
result

[['022PGL50022', u'24601675.jpg', 2],
 ['022PGL50022', u'109729316.jpg', 1],
 ['022PGL50022', u'5330005.jpg', 2],
 ['022PGL50022', u'82991702.jpg', 2],
 ['022PGL50022', u'82991672.jpg', 2],
 ['022PGL50022', u'5330007.jpg', 3],
 ['022PGL50022', u'24601151.jpg', 2],
 ['022PGL50022', u'60974412.jpg', 2],
 ['022PGL50022', u'82991634.jpg', 2],
 ['022PGL50022', u'63384957.jpg', 1],
 ['022PGL50022', u'82991671.jpg', 1],
 ['022PGL50022', u'60974434.jpg', 3],
 ['022PGL50022', u'5330011.jpg', 2],
 ['022PGL50022', u'82991694.jpg', 1],
 ['022PGL50022', u'24600027.jpg', 2],
 ['022PGL50022', u'60974397.jpg', 2],
 ['022PGL50022', u'82991656.jpg', 1],
 ['022PGL50022', u'63384956.jpg', 2],
 ['022PGL50022', u'24602678.jpg', 2],
 ['022PGL50022', u'107003917.jpg', 3],
 ['022PGL50022', u'82991711.jpg', 2],
 ['022PGL50022', u'82991713.jpg', 1],
 ['022PGL50022', u'82991690.jpg', 0],
 ['022PGL50022', u'5330009.jpg', 2],
 ['022PGL50022', u'82991681.jpg', 1],
 ['022PGL50022', u'75055600.jpeg', 1],
 ['022PGL5002

In [51]:
result = []
for i in range(len(a)):
    p = []
    p.append(l1[i][0])
    p.append(l1[i][1])
    p.append(int(pred[i]))
    result.append(p)

In [53]:
result

[['022PGL50022', u'24601675.jpg', 2],
 ['022PGL50022', u'109729316.jpg', 1],
 ['022PGL50022', u'5330005.jpg', 2],
 ['022PGL50022', u'82991702.jpg', 2],
 ['022PGL50022', u'82991672.jpg', 2],
 ['022PGL50022', u'5330007.jpg', 3],
 ['022PGL50022', u'24601151.jpg', 2],
 ['022PGL50022', u'60974412.jpg', 2],
 ['022PGL50022', u'82991634.jpg', 2],
 ['022PGL50022', u'63384957.jpg', 1],
 ['022PGL50022', u'82991671.jpg', 1],
 ['022PGL50022', u'60974434.jpg', 3],
 ['022PGL50022', u'5330011.jpg', 2],
 ['022PGL50022', u'82991694.jpg', 1],
 ['022PGL50022', u'24600027.jpg', 2],
 ['022PGL50022', u'60974397.jpg', 2],
 ['022PGL50022', u'82991656.jpg', 1],
 ['022PGL50022', u'63384956.jpg', 2],
 ['022PGL50022', u'24602678.jpg', 2],
 ['022PGL50022', u'107003917.jpg', 3],
 ['022PGL50022', u'82991711.jpg', 2],
 ['022PGL50022', u'82991713.jpg', 1],
 ['022PGL50022', u'82991690.jpg', 0],
 ['022PGL50022', u'5330009.jpg', 2],
 ['022PGL50022', u'82991681.jpg', 1],
 ['022PGL50022', u'75055600.jpeg', 1],
 ['022PGL5002

In [54]:
import os
import shutil

location = '/home/justdial/jd/Manyata/Dataset/try/testset'
for docID in os.listdir(location):
    folder_loc = os.path.join(location, docID)
    for image in os.listdir(folder_loc):
        for pred in result:
            if pred[1]==image:
                folder = pred[2]
                safe_mkdir(os.path.join(folder_loc, str(folder)))
                new_loc = os.path.join(str(folder), image)
                print (new_loc)      
                try:
                    os.rename(os.path.join(folder_loc, image), os.path.join(folder_loc, new_loc))
                    shutil.move(os.path.join(folder_loc, image), os.path.join(folder_loc, new_loc))
                except:
                    pass
                #print (folder)
    print ('docID "{}"completed'.format(docID))

2/24601675.jpg
1/109729316.jpg
2/5330005.jpg
2/82991702.jpg
2/82991672.jpg
3/5330007.jpg
2/24601151.jpg
2/60974412.jpg
2/82991634.jpg
1/63384957.jpg
1/82991671.jpg
3/60974434.jpg
2/5330011.jpg
1/82991694.jpg
2/24600027.jpg
2/60974397.jpg
1/82991656.jpg
2/63384956.jpg
2/24602678.jpg
3/107003917.jpg
2/82991711.jpg
1/82991713.jpg
0/82991690.jpg
2/5330009.jpg
1/82991681.jpg
1/75055600.jpeg
3/60974425.jpg
2/63384954.jpg
2/5330010.jpg
2/82991651.jpg
2/5330008.jpg
2/82991697.jpg
docID "022PGL50022"completed
2/101661171.jpg
3/101661167.jpg
1/112722356.jpg
2/86580349.jpg
1/33339009.png
2/101661169.jpg
1/80502619.jpg
1/80502621.jpg
2/112722355.jpg
1/115662008.jpg
2/101664238.jpg
1/80502626.jpg
1/83853365.jpg
2/80502632.jpg
2/33339012.png
2/101661158.jpg
3/101661166.jpg
2/101664241.jpg
2/101661164.jpg
1/101664240.jpg
3/101661168.jpg
2/101661165.jpg
2/101664158.jpg
2/112722352.jpg
2/101661157.jpg
1/33339006.png
1/80502634.jpg
2/101664239.jpg
1/83853358.jpg
2/112722348.jpg
1/112722353.jpg
1/8394883

2/80498683.jpg
2/25194086.jpg
2/94809713.jpg
2/85243571.jpg
2/80498715.jpg
2/84447278.jpg
2/83864285.jpg
2/85211196.jpg
2/115772287.jpg
3/25194080.jpg
2/107548163.jpg
2/100615489.jpg
2/96857053.jpg
2/101688723.jpg
2/87737540.jpg
2/81921832.jpg
2/89842272.jpg
2/92144871.jpg
2/82895158.jpg
2/98321382.jpg
2/91629158.jpg
2/83965935.jpg
2/96193782.jpg
2/106786279.jpg
2/100650280.jpg
2/108562762.jpg
3/25194072.jpg
2/114956373.jpg
2/84680161.jpg
2/80884860.jpg
2/81161150.jpg
2/116638963.jpg
2/25194068.jpg
1/102941710.jpg
2/82895159.jpg
2/80498718.jpg
2/102442576.jpg
2/80498752.jpg
2/106032956.jpg
2/88739263.jpg
2/80498724.jpg
0/25194057.jpg
2/80498698.jpg
2/93132046.jpg
2/86591527.jpg
2/96902392.jpg
2/97563575.jpg
2/82895160.jpg
2/88218710.jpg
2/95483096.jpg
2/80733458.jpg
2/80498770.jpg
2/87382189.jpg
2/105312057.jpg
2/109837719.jpg
2/112585178.jpg
2/85929081.jpg
2/83953215.jpg
2/111927853.jpg
2/100495089.jpg
2/26976889.jpg
2/100644383.jpg
2/81915360.jpg
2/113918230.jpg
2/87692972.jpg
2/9326

1/115896030.jpg
2/81209549.jpg
1/114936463.jpg
2/82891895.jpg
1/38643407.jpg
1/114925746.jpg
0/79391549.jpg
0/79391997.jpg
docID "022PK028607"completed
2/97611628.jpg
2/94286440.jpg
3/80495554.jpg
2/84008722.jpg
1/105218582.jpg
2/80625736.jpg
2/85206497.jpg
1/110114715.jpg
2/80495538.jpg
2/105020741.jpg
1/6980932.jpg
2/114936606.jpg
1/86582076.jpg
3/80822079.jpg
2/85123811.jpg
2/98321415.jpg
2/106276585.jpg
3/6980916.jpg
2/87809181.jpg
2/107659509.jpg
2/90908917.jpg
2/86582091.jpg
1/66589816.jpg
2/106057009.jpg
2/88057435.jpg
2/80690819.jpg
2/87364180.jpg
2/88297875.jpg
2/83949788.jpg
1/89317054.jpg
1/84613641.jpg
2/95483226.jpg
2/96548593.jpg
2/86903128.jpg
2/88993807.jpg
2/90381510.jpg
2/85317705.jpg
2/109193781.jpg
1/96857109.jpg
2/94466115.jpg
2/90105259.jpg
2/107004059.jpg
2/87458090.jpg
0/113918270.jpg
1/105020738.jpg
1/96333440.jpg
2/81802780.jpg
2/103258371.jpg
2/83052489.jpg
1/86582066.jpg
2/86990736.jpg
1/82891666.jpg
1/92385520.jpg
2/87655176.jpg
2/106648529.jpg
1/95757748.j

2/87365046.jpg
2/89316879.jpg
2/109625429.jpg
1/81932111.jpg
1/100489135.jpg
1/80595470.jpg
2/86590890.jpg
2/97611460.jpg
2/102941475.jpg
2/94123485.jpg
2/80738338.jpg
2/101872271.jpg
2/91493404.jpg
2/80625576.jpg
2/80738349.jpg
2/82894888.jpg
1/83863892.jpg
1/81937747.jpg
2/84615833.jpg
1/87725843.jpg
2/24578070.jpg
2/101872270.jpg
1/87365050.jpg
1/81935099.jpg
2/113900315.jpg
2/83863885.jpg
2/102121394.jpg
2/102963652.jpg
2/101872274.jpg
2/94809511.jpg
2/85794786.jpg
2/84615831.jpg
2/80502306.jpg
1/82887706.jpg
2/116549779.jpg
1/80502318.jpg
1/82894893.jpg
2/88993698.jpg
2/106056920.jpg
2/101872263.jpg
1/85128298.jpg
2/116334387.jpg
1/86590898.jpg
2/89740479.jpg
2/101872273.jpg
2/86590906.jpg
2/108062491.jpg
1/24578072.jpg
2/101872266.jpg
2/80502317.jpg
3/26917292.jpg
2/100637338.jpg
1/80502316.jpg
2/84867019.jpg
2/103908424.jpg
1/84615832.jpg
1/86590905.jpg
1/81907272.jpg
2/85462350.jpg
2/85243438.jpg
2/80884688.jpg
2/101872267.jpg
2/82894889.jpg
1/80642513.jpg
2/102909327.jpg
2/957

2/86919430.jpg
1/101855164.jpg
2/100650379.jpg
0/87485132.jpg
2/106276584.jpg
1/115896127.jpg
2/112725668.jpg
2/86919809.jpg
2/96548566.jpg
2/104724996.jpg
1/112725676.jpg
2/91394549.jpg
2/86071799.jpg
2/112725684.jpg
1/10795757.jpg
1/115210234.jpg
1/86632540.jpg
2/95251686.jpg
2/112725669.jpg
2/12164480.jpg
1/89817756.jpg
1/111020732.jpg
2/112725663.jpg
2/112725679.jpg
1/87717541.jpg
2/105120088.jpg
2/112725666.jpg
2/86156720.jpg
1/86071796.jpg
2/101478292.jpg
2/88400858.jpg
2/94286436.jpg
1/103909139.jpg
2/100496436.jpg
1/98271205.jpg
2/87809178.jpg
1/92385515.jpg
3/12164486.jpg
2/98321414.jpg
2/87809179.jpg
2/89477968.jpg
1/93455974.jpg
2/87683008.jpg
2/92144962.jpg
2/89132886.jpg
1/105218510.jpg
2/95977674.jpg
1/10795746.jpg
2/101688746.jpg
3/112725662.jpg
2/105020730.jpg
2/116464723.jpg
2/112725660.jpg
2/112725693.jpg
1/115662366.jpg
2/112725661.jpg
1/112725656.jpg
1/93261599.jpg
2/91024029.jpg
1/110680160.jpg
2/93902858.jpg
2/105218509.jpg
1/105020732.jpg
2/94466114.jpg
2/9609037

In [18]:
from PIL import Image

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def get_image_binary1(filename):
    try:
        #print (filename)
        image = Image.open(filename)        
        image = image.resize((53,53), Image.ANTIALIAS)
        #print (image)
        image = np.asarray(image, np.uint8)
        image = image.tobytes()
        #print (len(image))

        if (len(image) == 53*53*3):
            return image
    except exception as e:
        print ('Image Skipped! ...Error while reading "{}" '.format(filename))
        pass


def parse1(serialized):
    features = {
                'image' : tf.FixedLenFeature([], tf.string),
                'image_name': tf.FixedLenFeature([],tf.string)
               }
    parsed_example = tf.parse_single_example(serialized, features)
    image_raw = parsed_example['image']
    image_name = parsed_example['image_name']
    image = tf.decode_raw (image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, shape=[self.image_size,self.image_size,self.num_channels]) 
    return image, image_name

def load_tfrecord1(predict_file):
    tfrecord_name = predict_file.split('/')[-1]
    print ('Reading {} for prediction'.format(tfrecord_name))
    dataset = tf.data.TFRecordDataset(filenames = predict_file)
    dataset = dataset.map(parse1)
    total_prediction_examples = sum(1 for _ in tf.python_io.tf_record_iterator(predict_file))
    print ('{} files has found in {}'.format(total_prediction_examples, tfrecord_name))
    return (dataset, total_prediction_examples, tfrecord_name)

def write_tfrecord1(image, image_name):
    #print (image, image_name)
    a = get_image_binary1(image)
    print (len(a))
    example = tf.train.Example(features=tf.train.Features(feature = 
                                                              {                                                              
                                                                  'image': _bytes_feature(a),
                                                                  'image_name' : _bytes_feature(bytes(image_name, 'utf-8'))                                                            
                                                              }))
    print (example)
    return example


for docid in os.listdir('/home/justdial/jd/Manyata/Dataset/try/a'):
    #docid = folder_location.split('/')[-1]
    print ('creating TFRecord from {}'.format(docid))
    tfrecord_pred = 'tfrecord_test/'+docid
    print (tfrecord_pred)
    writer = tf.python_io.TFRecordWriter(tfrecord_pred)
    folder_location = os.path.join('/home/justdial/jd/Manyata/Dataset/try/a', docid)
    for images in os.listdir(folder_location):
        try:
            image_location = os.path.join(folder_location, images)
            #print (image_location)
            example = write_tfrecord1(image_location, images) 
            #print (type(example))
            writer.write(example.SerializeToString())
        except Exception as e:
            print ('Image Skipped!!!! ...Error while reading "{}" '.format(images))
            pass
    writer.close() 

creating TFRecord from other(no_ambience)
tfrecord_test/other(no_ambience)
8427
Image Skipped!!!! ...Error while reading "sai-bikaner-sweets-corner-devli-gaon-delhi-o5ghd.jpg" 
8427
Image Skipped!!!! ...Error while reading "1436767.jpg" 
8427
Image Skipped!!!! ...Error while reading "image_17850.jpg" 
8427
Image Skipped!!!! ...Error while reading "1166745.jpg" 
8427
Image Skipped!!!! ...Error while reading "1284921.jpg" 
8427
Image Skipped!!!! ...Error while reading "1255354.jpg" 
8427
Image Skipped!!!! ...Error while reading "1256937.jpg" 
8427
Image Skipped!!!! ...Error while reading "626611.jpg" 
8427
Image Skipped!!!! ...Error while reading "image_14366.jpg" 
8427
Image Skipped!!!! ...Error while reading "image_15505.jpg" 
8427
Image Skipped!!!! ...Error while reading "image_16658.jpg" 
8427
Image Skipped!!!! ...Error while reading "1907282.jpg" 
8427
Image Skipped!!!! ...Error while reading "1491843.jpg" 
8427
Image Skipped!!!! ...Error while reading "1492529.jpg" 
8427
Image Skip

8427
Image Skipped!!!! ...Error while reading "the-bayview-hotel-marine-plaza--new-marine-lines-churchgate-mumbai-continental-restaurants-bl9b2.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-bark-teynampet-chennai-restaurants-t2nz626a60.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-99-menu-vasundhara-enclave-delhi-north-indian-restaurants-ruhg3zkpxh.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-bayview-hotel-marine-plaza-new-marine-lines-churchgate-mumbai-restaurants-3i5kcnl.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-bayview-hotel-marine-plaza--new-marine-lines-churchgate-mumbai-continental-restaurants-r0lyy.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-avenue-kandivali-east-mumbai-restaurants-and-bars-uq3wp.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-cake-express-tiruchirappalli-cantt-trichy-bakeries-whz6jkcqqx.jpg" 
8427
Image Skipped!!!! ...Error while reading "the-avenue-kandivali-east-mumbai-restaurants-and-bars

In [30]:
example = tf.train.Example(features=tf.train.Features(feature = 
                                                              {                                                              
                                                                  'image': _bytes_feature(b),
                                                                  'image_name' : _bytes_feature(bytes(image_name))                                                            
                                                              }))

In [24]:
l = '/home/justdial/jd/Manyata/Dataset/try/a/other(nodrink)/99768816.jpg'
image_name = '99768816.jpg'

In [25]:
b = get_image_binary1(l)

In [2]:
import os
sample_location = '/home/justdial/jd/Manyata/Dataset/FvOnFDvOnAvO/dataset'
a = os.listdir(sample_location)

In [4]:
ambience = os.listdir(os.path.join(sample_location, a[2]))

In [8]:
len(ambience)

11102